In [1]:

import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread


import speech_recognition as sr
import os ,time
import voice
from typing import Dict, List
import numpy as np
import pyaudio

# transcriberregTrans=voice.WhisperRegTrnscriber()

messages = Queue()

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Audio settings
STEP_IN_SEC: int = 1    # We'll increase the processable audio data by this
LENGHT_IN_SEC: int = 6    # We'll process this amount of audio data together maximum
NB_CHANNELS = 1
RATE = 16000
CHUNK = RATE
# Visualization (expected max number of characters for LENGHT_IN_SEC audio)
MAX_SENTENCE_CHARACTERS = 80

CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2


stats: Dict[str, List[float]] = {"overall": [], "transcription": [], "postprocessing": []}


def callbackFunc(recognizer,audio):                          # this is called from the background thread
    

    try:
        
        transcription_start_time = time.time()
        # transcription,_ =transcriberregTrans.transcribeLang(audio.)
        transcription=recognizer.recognize_whisper(audio,language='he')
        # print(transcription)
        transcription_end_time = time.time()

        # transcription = " ".join(segments)
        
        # remove anything from the text which is between () or [] --> these are non-verbal background noises/music/etc.
        # transcription = re.sub(r"\[.*\]", "", transcription)
        # transcription = re.sub(r"\(.*\)", "", transcription)
        # We do this for the more clean visualization (when the next transcription we print would be shorter then the one we printed)
        transcription = transcription.ljust(MAX_SENTENCE_CHARACTERS, " ")

        transcription_postprocessing_end_time = time.time()
        print(transcription)
        with output:
            display(transcription)
        # output.append_stdout(transcription)

        # print(transcription, end='\r', flush=True)

        overall_elapsed_time = transcription_postprocessing_end_time - transcription_start_time
        transcription_elapsed_time = transcription_end_time - transcription_start_time
        postprocessing_elapsed_time = transcription_postprocessing_end_time - transcription_end_time
        stats["overall"].append(overall_elapsed_time)
        stats["transcription"].append(transcription_elapsed_time)
        stats["postprocessing"].append(postprocessing_elapsed_time)

                
                # print("You said " + recognizer.recognize_whisper(audio,language='he'))  # received audio data, now need to recognize it
    except LookupError:
        # with output:
        #     display("Oops! Didn't catch that")
        print("Oops! Didn't catch that")



def start_voice_r():
    
    print("Starting")
    # with output:
    #         display("Starting...")
            
    if  messages.empty():
        messages.put(True)
    
    
        r = sr.Recognizer()
        m = sr.Microphone()
        with m as source:
            # r.energy_threshold = 270
            
            r.pause_threshold = 0.8  # seconds of non-speaking audio before a phrase is considered complete 
            r.phrase_threshold = 0.3  # minimum seconds of speaking audio before we consider the speaking audio a phrase - values below this are ignored (for filtering out clicks and pops)
            r.non_speaking_duration = 0.4  # seconds of non-speaking audio to keep on both sides of the recording
            r.dynamic_energy_threshold = True
            r.adjust_for_ambient_noise(source)  # we only need to calibrate once, before we start listening

        # start listening in the background (note that we don't have to do this inside a `with` statement)
        stop_listening = r.listen_in_background(source=m, callback=callbackFunc,phrase_time_limit=LENGHT_IN_SEC)

    
        with output:
            display("Starting...")
            
        while not messages.empty():
            time.sleep(1)   
    
        stop_listening()
        print('stoped')
    else:
        pass    


pygame 2.5.2 (SDL 2.28.3, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:

output = widgets.Output()


record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)




def start_recording(data):
    
    record = Thread(target=start_voice_r)
    record.start()
        
    # with output:
    #     display("Starting...")
            
        

def stop_recording(data):
    messages.get_nowait()
    messages.task_done()
    # with output:
        # display("Stopping.")
        
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

pygame 2.5.2 (SDL 2.28.3, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

Starting


 פשוט חייה                                                                      


 רמצו, מה besser בהלנים? האם? זהו גם מרובה בכלל                                 


stoped
